In [ ]:
from os import listdir, makedirs
from os.path import isfile, join, exists
import pandas as pd
import numpy as np

old_basepath = '../basedata/PCD Data/Data before 2020-9'
new_basepath = '../basedata/PCD Data/Data after 2020-7/PCD data after 2020-7.csv'

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import os 
import os.path as osp 
from collections import defaultdict
from PIL import Image
import math 
from tqdm.notebook import tqdm 
import random

import torch 
from torch import nn 
import torch.nn.functional as F 
from torch import optim 
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import datetime


In [ ]:
station_lat_long = pd.read_csv('prepared_data/others/station_lat_long.csv', usecols=['stationIDs', 'lats', 'longs'])

In [ ]:
station_lat_long

In [ ]:
station

In [ ]:
listdir(data_base_path)[0]

In [ ]:
station_lat_long.iloc[1]

In [251]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
import time

def scrape(url):
    # url = f'https://earth.nullschool.net/chem/surface/level/anim=off/overlay=so2smass/equirectangular/loc={lng},{lat}'
    #go to web/#current
    driver.get(url=url)
    element = WebDriverWait(driver,9999).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="spotlight-panel"]/div[3]/div')))
    data_status = driver.find_element(By.XPATH,'/html/body/main/div[3]/div[1]/div')
    if data_status.text=="Downloading...":
        while True:
            time.sleep(0.05)
            data_status = driver.find_element(By.XPATH,'/html/body/main/div[3]/div[1]/div')
            if data_status.text=="Downloading...":
                continue
            else :
                break
    #so2
    data = element.text.split(' ')[0]
    return data



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/grniss/.wdm/drivers/chromedriver/mac64_m1/101.0.4951.41/chromedriver] found in cache


In [252]:
from os import listdir
from os.path import join
import pandas as pd
import pytz

station_lat_long = pd.read_csv('prepared_data/others/station_lat_long.csv', usecols=['stationIDs', 'lats', 'longs'])
data_base_path = 'prepared_data/stations'
to_use_base_path = 'prepared_data/to_uses'
scraped_base_path = 'prepared_data/scraped'
start = 0
stop = 1
for i in range(start, stop):
    cur_station = station_lat_long.iloc[i]
    fn = f'{cur_station.stationIDs}.csv'
    df = pd.read_csv(join(data_base_path, fn)).set_index('datetime')
    to_use = pd.read_csv(join(to_use_base_path, fn)).set_index('datetime')
    df = df.loc[to_use.index]
    df['set_no'] = to_use['set_no']
    df.index = pd.to_datetime(df.index , format='%Y-%m-%d %H:%M:%S%z')
    for datetime in df.index:
        datetime_utc = datetime.astimezone(pytz.utc)
        lat = station_lat_long.iloc[i]['lats']
        lng = station_lat_long.iloc[i]['longs']
        year = datetime_utc.year
        month = datetime_utc.month
        day = datetime_utc.day
        hour = datetime_utc.hour
        urls = {
            'PM25':f'https://earth.nullschool.net/#{year}/{month}/{day}/{hour}00Z/particulates/surface/level/anim=off/overlay=pm2.5/equirectangular/loc={lng},{lat}',
            'PM10':f'https://earth.nullschool.net/#{year}/{month}/{day}/{hour}00Z/particulates/surface/level/anim=off/overlay=pm10/equirectangular/loc={lng},{lat}',
            'NO2':f'https://earth.nullschool.net/#{year}/{month}/{day}/{hour}00Z/chem/surface/level/anim=off/overlay=no2/equirectangular/loc={lng},{lat}',
            'SO2':f'https://earth.nullschool.net/#{year}/{month}/{day}/{hour}00Z/chem/surface/level/anim=off/overlay=so2smass/equirectangular/loc={lng},{lat}',
            'CO':f'https://earth.nullschool.net/#{year}/{month}/{day}/{hour}00Z/chem/surface/level/anim=off/overlay=cosc/equirectangular/loc={lng},{lat}',
            'Rain':f'https://earth.nullschool.net/#{year}/{month}/{day}/{hour}00Z/wind/surface/level/anim=off/overlay=relative_humidity/equirectangular/loc={lng},{lat}',
            'Temp':f'https://earth.nullschool.net/#{year}/{month}/{day}/{hour}00Z/wind/surface/level/anim=off/overlay=temp/equirectangular/loc={lng},{lat}'
        }
        cur_row = df.loc[datetime]
        na_checker = cur_row.isna()
        for col in cur_row.index:
            if na_checker[col] and col in urls:
                data = scrape(urls[col])
                df.loc[datetime, col] = data
    df.to_csv(join(scraped_base_path, fn))
                
            
    
    

    

datetime 2021-07-18 00:00:00+07:00
col Temp
data 21.3
datetime 2021-07-18 03:00:00+07:00
col Temp
data 21.3
datetime 2021-07-18 06:00:00+07:00
col Temp
data 21.2
datetime 2021-07-18 09:00:00+07:00
col Temp


KeyboardInterrupt: 

In [249]:
for col in cur_row.index:
            if na_checker[col] and col in urls:
                print(col)

Temp
Rain
SO2


In [246]:
from os import listdir
from os.path import join
import pandas as pd

data_base_path = 'prepared_data/stations'
to_use_base_path = 'prepared_data/to_uses'
count = 0
start = 0
i = 0
for station in listdir(data_base_path):
    if start <= i:
        df = pd.read_csv(join(data_base_path, station))
        to_use = pd.read_csv(join(to_use_base_path, station))
        df = df.set_index('datetime').loc[to_use['datetime']]
        count += df[['CO', 'NO2', 'SO2', 'O3', 'PM10', 'Temp', 'Rain', 'PM25']].isna().sum().sum()
        # for col in df.columns:
        #     for dt in df[col][df[col].isna()]:
        #         scrape(lat, long, datetime, col)
    i += 1
    # if((df.isna().sum() > 0).sum() > 1):
    #     print((df.isna().sum() > 0).sum() > 1)
    #     break
count

865272

In [ ]:
import pytz
pd.to_datetime(df.index, format='%Y-%m-%d %H:%M:%S%z')[0].astimezone(pytz.utc).day

In [ ]:
pd.to_datetime(df.index, format='%Y-%m-%d %H:%M:%S%z')[0].astimezone(pytz.utc).hour

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)